# AI To Play Mine Sweeper

## The Game Object

I developed the pythonic minesweeper. It is unit tested fairly throughly; up until visualizations and tracking features of the project present themselves. Note: The 9's are bombs; because it is impossible for a cell to be sourounded by more Bombs then there are cells surronding it.

In [7]:
import game
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from ipywidgets import IntProgress
from IPython.display import display
import random

In [2]:
mygame = game.minesweeper(14, 8, 25)


for row in mygame.board:
    for c in row:
        print(c, "\t", end="")
    print("\n")
#mygame.clickCell((0,0))

9.0 	9.0 	3.0 	2.0 	2.0 	2.0 	9.0 	1.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	

3.0 	9.0 	3.0 	9.0 	9.0 	3.0 	2.0 	1.0 	0.0 	0.0 	1.0 	1.0 	1.0 	0.0 	

2.0 	2.0 	3.0 	3.0 	4.0 	9.0 	1.0 	0.0 	0.0 	0.0 	1.0 	9.0 	2.0 	1.0 	

9.0 	1.0 	1.0 	9.0 	2.0 	1.0 	1.0 	0.0 	0.0 	1.0 	2.0 	3.0 	3.0 	9.0 	

2.0 	2.0 	1.0 	2.0 	2.0 	2.0 	1.0 	2.0 	2.0 	4.0 	9.0 	3.0 	9.0 	2.0 	

9.0 	2.0 	0.0 	2.0 	9.0 	3.0 	9.0 	2.0 	9.0 	9.0 	9.0 	4.0 	2.0 	2.0 	

9.0 	3.0 	1.0 	2.0 	9.0 	3.0 	1.0 	3.0 	4.0 	9.0 	3.0 	2.0 	9.0 	1.0 	

2.0 	9.0 	1.0 	1.0 	1.0 	1.0 	0.0 	1.0 	9.0 	2.0 	1.0 	1.0 	1.0 	1.0 	



## Visualizing the Game

The following is a rough gui for the game. Feel free to mess around; when you loose all of the cells will disable and you will be alerted. When you win you will be alerted. Note: If you want to start a new game you will have to run the cell above because you need to make a new object to start over

In [3]:
mygame.playOnNoteBook()

## Neruel Network to Play Mine Sweeper (Classification Approach) 

We will attempt to play minesweeper by generating lots of games (which will be used as our data). 

Things that we'll need to do:

Pad the game board with some element.
This way we distiguish the edges of the game board (all of our data needs to have the same number of features) 

Represent "unknown squares" in our classification we will have some data that are "completed games" (the whole board is visible) and others where the game is 80% finished; 60%; 40% and 20%. (Our "Solve Percent" utility will help us with this)

In [4]:
mygame = game.minesweeper(14, 8, 25)
mygame.solvePercent(.2)
mygame.playOnNoteBook()

In [6]:
theData = []
N = 2 #N represents the number of surronding squares to get


for p in [.8, .7, .6, .5, .4, .3, .2, .1]:
    for i in range(1500):
        bomb_ratio = random.uniform(.1, .25)
        width = random.randint(8,16)
        height = random.randint(8,16)
        mygame = game.minesweeper(width, height, height*width*bomb_ratio)
        mygame.solvePercent(p)
        dataToAdd = np.pad(mygame.visible, (N), 'constant', constant_values=(-1))
        dataToAdd = np.where(dataToAdd=='*', 0, dataToAdd)
        theData.append(dataToAdd)
        
for i in range(1500):
    mygame = game.minesweeper(16, 30, 99)
    dataToAdd = np.pad(mygame.board, (N), 'constant', constant_values=(-1))
    theData.append(dataToAdd.astype('<U3'))


In [10]:
theRealData = []


for data in theData:
    innerData = data[N:-N, N:-N]
    for index, x in np.ndenumerate(innerData):
        toAppend = []
        toAppend.append(x)
        for i in range((2*N+1)**2):
            if i != (((2*N+1)**2)//2):
                a = i//(2*N+1)
                b = i%(2*N+1)
                toAppend.append(data[index[0] + a, index[1] + b])
        theRealData.append(toAppend)

(Comment this line out if you don't want to eliminate the unknown value prediction)

In [11]:
theRealData = np.array(theRealData)
theRealData = theRealData[theRealData[:, 0] != '?']

MemoryError: 

This next line will make it so that it's classifying not bomb (which is true) and bomb (false) -- so if a cell is a bomb it will give it a false value. Think of it as an "okay to click" classification.

In [ ]:
theRealData[:, 0] = ['1.0' if r else '0.0' for r in (theRealData[:, 0] != '9.0')]
theRealData

In [ ]:
theRealData = np.where(theRealData=='?', 10, theRealData)
X, y = theRealData[:, 1:].astype(float).astype(str), theRealData[:, 0].astype(float).astype(str)

enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(X)

In [ ]:
#print([cat.shape for cat in enc.categories_])
print(enc.categories_)
X = enc.transform(X).toarray()

In [ ]:
X.shape

(1754147, 264)

In [ ]:
y.shape

(1754147,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
"""unique, counts = np.unique(y_train, return_counts=True)
#print(unique, counts)
oneindes = np.where(y_train=='1.0')[0]
np.random.shuffle(oneindes)
newTrainIndies = np.append(oneindes[:counts[0]], np.where(y_train=='0.0')[0])
#np.unique(y_train[newTrainIndies], return_counts=True)
y_train = y_train[newTrainIndies]
X_train = X_train[newTrainIndies]"""

"unique, counts = np.unique(y_train, return_counts=True)\n#print(unique, counts)\noneindes = np.where(y_train=='1.0')[0]\nnp.random.shuffle(oneindes)\nnewTrainIndies = np.append(oneindes[:counts[0]], np.where(y_train=='0.0')[0])\n#np.unique(y_train[newTrainIndies], return_counts=True)\ny_train = y_train[newTrainIndies]\nX_train = X_train[newTrainIndies]"

The Hidden Layer "24" is inspired by the idea that each surrounding node has been one-hot-encoded into 12 features

In [ ]:
clf = MLPClassifier(solver='adam', activation="tanh", alpha=1e-5, hidden_layer_sizes=(100, 100), verbose=True)

In [ ]:
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Iteration 1, loss = 0.07617874
Iteration 2, loss = 0.04858983
Iteration 3, loss = 0.04465398
Iteration 4, loss = 0.04235168
Iteration 5, loss = 0.04081377
Iteration 6, loss = 0.03954967
Iteration 7, loss = 0.03855964
Iteration 8, loss = 0.03761411
Iteration 9, loss = 0.03667858
Iteration 10, loss = 0.03594042
Iteration 11, loss = 0.03516730
Iteration 12, loss = 0.03449253
Iteration 13, loss = 0.03379877
Iteration 14, loss = 0.03324135
Iteration 15, loss = 0.03255773
Iteration 16, loss = 0.03213016
Iteration 17, loss = 0.03150006
Iteration 18, loss = 0.03102578
Iteration 19, loss = 0.03055755
Iteration 20, loss = 0.03004622
Iteration 21, loss = 0.02961723
Iteration 22, loss = 0.02915911
Iteration 23, loss = 0.02880253
Iteration 24, loss = 0.02833926
Iteration 25, loss = 0.02802860
Iteration 26, loss = 0.02767822
Iteration 27, loss = 0.02718538
Iteration 28, loss = 0.02692918
Iteration 29, loss = 0.02655132
Iteration 30, loss = 0.02635415
Iteration 31, loss = 0.02591840
Iteration 32, los

In [ ]:
clf.predict_proba(X_test[0].reshape(1, -1))[0][1]

# The AI that Plays Mine Sweeper

In [ ]:
class getBoundry:
    def __init__(self):
        self.visited = []
        self.toReturn = []
        self.perim = []
    def getBoundry(self, gameObj, index):
        if gameObj.visible[index[0]][index[1]] !="?" and index not in self.visited:
            self.visited.append(index)
            self.toReturn.append(index)
            for i in [self.getBoundry(gameObj, i) for i in gameObj.getSurroundingSquares(index)]:
                gameObj.getSurroundingSquares(index)
        elif index not in self.visited and (index[0] <= gameObj.width and index[1] <= gameObj.height):
            self.visited.append(index)
            self.toReturn.append(index)
            self.perim.append(index)
            for j in gameObj.getSurroundingSquares(index):
                if j not in self.toReturn:
                    self.perim.append(j)
        return self.toReturn

    
class AIPlayer():
    def __init__(self, mygame, watch=False, initClick=(1,1), bombGuessTol = .85, wait=False):
        if watch:
            mygame.playOnNoteBook()

        #Our algorithm selects (0,0) first every time any way -- input is all the same on all unknown...
        #To test if there is a better place than this think about classifying better starting points... Differn't shapes...

        if watch:
            mygame.gameboard[initClick[0]][initClick[1]].click()
        else:
            mygame.clickCell(initClick)

        data = np.array(mygame.visible).astype('<U3')
        data = np.pad(data, (N), 'constant', constant_values=(-1))
        self.data = np.where(data=='*', 0, data)
        innerData = data[N:-N, N:-N]

        while ("?" in " ".join([" ".join(map(str, row)) for row in mygame.visible]) and not wait):
            curbest = 0
            bestindex = (0, 0)
            data = np.array(mygame.visible).astype('<U3')
            data = np.pad(data, (N), 'constant', constant_values=(-1))
            self.data = np.where(data=='*', 0, data)
            boundObject = getBoundry()
            boundObject.getBoundry(mygame, initClick)
            boundry = list(filter(lambda x: mygame.visible[x[0]][x[1]]=="?", sorted(list(set(boundObject.perim)))))
            proba = self.getProba(boundry)
            bestindex = boundry[np.argmax(proba[:, 1])]
            for bombGuess in np.where(proba[:,0] > bombGuessTol)[0]:
                bombGuessIndex = boundry[bombGuess]
                mygame.flag(bombGuessIndex)
                if watch:
                    mygame.gameboard[bombGuessIndex[0]][bombGuessIndex[1]].description = "9.0"
            if watch:
                mygame.gameboard[bestindex[0]][bestindex[1]].click()
            else:
                mygame.clickCell(bestindex)
                
    def getProba(self, boundry):
        toReturn = []
        for index in boundry:
            toAppend = []
            for i in range((2*N+1)**2):
                if i != (((2*N+1)**2)//2):
                    a = i//(2*N+1)
                    b = i%(2*N+1)
                    toAppend.append(self.data[index[0] + a, index[1] + b])
            toReturn.append(toAppend)
        toReturn = np.array(toReturn)
        toReturn = np.where(toReturn=='?', 10, toReturn)
        toReturn = toReturn.astype(float).astype(str)
        #toReturn = toReturn.reshape(1,-1)
        toReturn = enc.transform(toReturn).toarray()
        return clf.predict_proba(toReturn)

        

In [ ]:
wins = 0
loses = 0
gamesToPlay = 1000

f = IntProgress(min=0, max=gamesToPlay) # instantiate the bar
display(f) # display the bar
lossGames = []

for _ in range(gamesToPlay):
    f.value += 1
    mygame = game.minesweeper(9, 9, 10, record=True)
    AIPlayer(mygame, bombGuessTol=.9)
    if mygame.visible == "You have lost":
        loses += 1
        lossGames.append(mygame)
    else:
        wins += 1

print("The AI won {} times and lose {} times".format(wins, loses))        
    
#print(mygame.visible)

In [ ]:
print(sorted(lossGames[1].gameRecord.items()))

In [ ]:
lossGames[1].seeRecordOnNotebook()

In [ ]:
lossGames[0].visible

In [ ]:
lossGames[0].board

In [ ]:
boundObject = getBoundry()
boundObject.getBoundry(lossGames[0], (1,1))
boundry = list(filter(lambda x: lossGames[0].visible[x[0]][x[1]]=="?", sorted(list(set(boundObject.perim))))) + [(5,4)]

myAI = AIPlayer(lossGames[0], watch=True, wait=True)

In [ ]:
for i, proba in enumerate(myAI.getProba(boundry)):
    print(boundry[i], proba)

#mygame = game.minesweeper(30, 16, 99, record=True)
#AIPlayer(mygame, watch=True)

In [ ]:
myAI = AIPlayer(lossGames[1], watch=True)

In [ ]:
boundry

# Idea's to Improve:
### Mess with Hyper Params
### Mess with the solve function (make exact percentages etc)
### Mess with what we are classifying (We just want to know if something is a bomb or not...
### More Data surronding each point
### All unique Data

## Oberservations
Just because a classifier is good doesn't make it good for the game

# Messing with Hyper Params

In [ ]:
#clf2 = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

#clf2.fit(X_train, y_train)

#y_pred = clf2.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [ ]:
# This one is inspired by the one hot incoding (Each feature was split into 11 parts)
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(11), random_state=1)

#clf3.fit(X_train, y_train)

#y_pred = clf3.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [ ]:
#import pickle

#filename = 'clf1.sav'
#pickle.dump(clf, open(filename, 'wb'))

#filename = 'clf2.sav'
#pickle.dump(clf2, open(filename, 'wb'))

#filename = 'clf3.sav'
#pickle.dump(clf3, open(filename, 'wb'))

In [ ]:
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(8, 3), random_state=1)

#clf3.fit(X_train, y_train)

#y_pred = clf3.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [ ]:
#filename = 'clf3.sav'
#pickle.dump(clf3, open(filename, 'wb'))

# Messing with what we are classifying

In [ ]:
np.unique(y.shape)